In [1]:
import pandas as pd
import sqlite3


The [sf-salaries dataset](https://www.kaggle.com/kaggle/sf-salaries) is provided by Kaggle and has a CC0: Public Domain licence.


The SQLitetutorial-example-database is provided by sqlitetutorial.net and can be found [here](https://www.sqlitetutorial.net/sqlite-sample-database/)

Connect to the sf-salaries database

In [6]:
conn = sqlite3.connect("""./data/database.sqlite""")

Use read_sql of pandas to treat the database as a pandas dataframe

In [7]:
df = pd.read_sql("SELECT * FROM salaries LIMIT 10;",conn)
df.head(10)

,Id,EmployeeName,JobTitle,BasePay,OvertimePay,OtherPay,Benefits,TotalPay,TotalPayBenefits,Year,Notes,Agency,Status
0,1,NATHANIEL FORD,GENERAL MANAGER-METROPOLITAN TRANSIT AUTHORITY,167411.18,0.00,400184.25,,567595.43,567595.43,2011,,San Francisco,
1,2,GARY JIMENEZ,CAPTAIN III (POLICE DEPARTMENT),155966.02,245131.88,137811.38,,538909.28,538909.28,2011,,San Francisco,
2,3,ALBERT PARDINI,CAPTAIN III (POLICE DEPARTMENT),212739.13,106088.18,16452.60,,335279.91,335279.91,2011,,San Francisco,
3,4,CHRISTOPHER CHONG,WIRE ROPE CABLE MAINTENANCE MECHANIC,77916.00,56120.71,198306.90,,332343.61,332343.61,2011,,San Francisco,
4,5,PATRICK GARDNER,"DEPUTY CHIEF OF DEPARTMENT,(FIRE DEPARTMENT)",134401.60,9737.00,182234.59,,326373.19,326373.19,2011,,San Francisco,
5,6,DAVID SULLIVAN,ASSISTANT DEPUTY CHIEF II,118602.00,8601.00,189082.74,,316285.74,316285.74,2011,,San Francisco,
6,7,ALSON LEE,"BATTALION CHIEF, (FIRE DEPARTMENT)",92492.01,89062.90,134426.14,,315981.05,315981.05,2011,,San Francisco,
7,8,DAVID KUSHNER,DEPUTY DIRECTOR OF INVESTMENTS,256576.96,0.00,51322.50,,307899.46,307899.46,2011,,San Francisco,
8,9,MICHAEL MORRIS,"BATTALION CHIEF, (FIRE DEPARTMENT)",176932.64,86362.68,40132.23,,303427.55,303427.55,2011,,San Francisco,
9,10,JOANNE HAYES-WHITE,"CHIEF OF DEPARTMENT, (FIRE DEPARTMENT)",285262.00,0.00,17115.73,,302377.73,302377.73,2011,,San Francisco,


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Id                10 non-null     int64  
 1   EmployeeName      10 non-null     object 
 2   JobTitle          10 non-null     object 
 3   BasePay           10 non-null     float64
 4   OvertimePay       10 non-null     float64
 5   OtherPay          10 non-null     float64
 6   Benefits          10 non-null     object 
 7   TotalPay          10 non-null     float64
 8   TotalPayBenefits  10 non-null     float64
 9   Year              10 non-null     int64  
 10  Notes             10 non-null     object 
 11  Agency            10 non-null     object 
 12  Status            10 non-null     object 
dtypes: float64(5), int64(2), object(6)
memory usage: 1.1+ KB


Using COUNT(*) and DISTINCT


In [9]:
pd.read_sql("SELECT COUNT(*) FROM salaries LIMIT 10;",conn)

,COUNT(*)
0,148654


In [10]:
pd.read_sql("SELECT DISTINCT Year FROM salaries;",conn)

,Year
0,2011
1,2012
2,2013
3,2014


Store the query in its own object and pass the object into the pd.read_sql() method. This makes the code reusableand a little more readable. 

In [11]:
query_agency = """
SELECT DISTINCT agency
FROM
    salaries
LIMIT 10;"""

pd.read_sql(query_agency, conn)

,Agency
0,San Francisco


In [12]:
query_notes = """
SELECT DISTINCT notes
FROM
    salaries
LIMIT 10;"""

pd.read_sql(query_notes, conn)

,Notes
0,


In [13]:
query_distinct_jobs = """
SELECT 
    COUNT(DISTINCT JobTitle) AS Unique_Jobs
FROM
    salaries
LIMIT 10;
"""
pd.read_sql(query_distinct_jobs, conn)

,Unique_Jobs
0,2159


Displayed the top 20 highest paid jobs.

In [14]:
query_avg_totalpaybenefits = """
SELECT 
    JobTitle, COUNT(JobTitle) Job_Count, AVG(TotalPayBenefits) AVG_TOT_PAY_BENE, AVG(TotalPay) AVG_TOT_PAY, MAX(TotalPayBenefits) MAX_TOT_PAY_BENE, MIN(TotalPayBenefits) MIN_TOT_PAY_BENE
FROM
    salaries
WHERE
    TotalPayBenefits IS NOT NULL AND TotalPayBenefits != 0
GROUP BY
    JobTitle
ORDER BY
    AVG(TotalPayBenefits) DESC
LIMIT 20;
"""
pd.read_sql(query_avg_totalpaybenefits, conn)

,JobTitle,Job_Count,AVG_TOT_PAY_BENE,AVG_TOT_PAY,MAX_TOT_PAY_BENE,MIN_TOT_PAY_BENE
0,Chief Investment Officer,1,436224.360000,339653.700000,436224.36,436224.36
1,Chief of Police,3,411732.266667,329183.646667,425815.28,391362.30
2,"Chief, Fire Department",3,408865.326667,325971.683333,422353.40,386807.48
3,GENERAL MANAGER-METROPOLITAN TRANSIT AUTHORITY,2,399211.275000,399211.275000,567595.43,230827.12
4,"Gen Mgr, Public Trnsp Dept",3,380696.440000,297769.413333,386168.49,376002.11
5,"Dep Dir for Investments, Ret",2,355731.260000,278592.545000,398421.67,313040.85
6,Mayor,3,354212.906667,275852.530000,364814.51,335272.55
7,"Adm, SFGH Medical Center",3,347079.706667,270047.446667,352343.22,339368.15
8,Controller,3,343061.140000,264750.753333,347713.88,336683.28
9,Asst Chf of Dept (Fire Dept),18,342597.550556,281763.483889,396778.68,293038.56


In [ ]:
query_count_jobs = """
SELECT 
    JobTitle, COUNT(JobTitle) Job_Count, AVG(TotalPayBenefits) AVG_TOT_PAY_BENE, AVG(TotalPay) AVG_TOT_PAY
FROM
    salaries
WHERE
    TotalPayBenefits IS NOT NULL AND TotalPayBenefits != 0
GROUP BY
    JobTitle
ORDER BY
    COUNT(JobTitle) DESC
LIMIT 20;
"""
pd.read_sql(query_count_jobs, conn)

Using keywords having, union, and intersect

In [15]:
query_count_jobs_two = """
SELECT 
    JobTitle, COUNT(JobTitle) Job_Count, AVG(TotalPayBenefits) AVG_TOT_PAY_BENE, AVG(TotalPay) AVG_TOT_PAY
FROM
    salaries
WHERE
    TotalPayBenefits IS NOT NULL AND TotalPayBenefits != 0
GROUP BY
    JobTitle
HAVING
    COUNT(JobTitle) BETWEEN 500 AND 1000
ORDER BY
    COUNT(JobTitle) DESC
LIMIT 20;
"""
pd.read_sql(query_count_jobs_two, conn)

,JobTitle,Job_Count,AVG_TOT_PAY_BENE,AVG_TOT_PAY
0,EMT/Paramedic/Firefighter,918,167899.182614,133186.602092
1,Gardener,894,79951.820358,56127.486913
2,Parking Control Officer,866,80594.730058,57412.047841
3,Senior Clerk,840,65062.688857,44217.102226
4,Library Page,799,31954.821477,21942.882628
5,CUSTODIAN,796,40521.552776,40521.552776
6,FIREFIGHTER,794,139292.045945,139292.045945
7,POLICE OFFICER III,779,127834.395918,127834.395918
8,Clerk,764,52924.187762,36131.890288
9,Eligibility Worker,760,68343.523158,48232.818974


Using UNION, we can append rows of queries with matching columns and datatypes.

In [16]:
query_union = """
SELECT jobtitle, 
    COUNT(jobtitle) AS Count
FROM 
    salaries 
WHERE 
    jobtitle LIKE '%officer' AND
    Year = 2011
GROUP BY 
    jobtitle
                        
                        
UNION
                 
                 
SELECT jobtitle, 
    COUNT(jobtitle) AS Count
FROM 
    salaries 
WHERE 
    jobtitle LIKE 'police%' AND
    Year = 2011
GROUP BY 
    jobtitle 
ORDER BY 
    Count DESC
LIMIT 10;"""
pd.read_sql(query_union, conn)

,JobTitle,Count
0,POLICE OFFICER III,779
1,POLICE OFFICER I,467
2,POLICE OFFICER II,361
3,PARKING CONTROL OFFICER,272
4,DEPUTY PROBATION OFFICER,136
5,BUILDING AND GROUNDS PATROL OFFICER,49
6,SENIOR COLLECTIONS OFFICER,44
7,AIRPORT SAFETY OFFICER,40
8,SENIOR PARKING CONTROL OFFICER,26
9,TRAINING OFFICER,20


INTERSECT returns the matching rows found in both queries.

In [17]:
query_intersect_union = """
SELECT jobtitle, 
    COUNT(jobtitle) AS Count
FROM 
    salaries 
WHERE 
    jobtitle LIKE '%officer%' AND
    Year = 2011
GROUP BY 
    jobtitle
                        
                        
INTERSECT
                 
                
SELECT jobtitle, 
    COUNT(jobtitle) AS Count
FROM 
    salaries 
WHERE 
    jobtitle LIKE '%police%' AND
    Year = 2011
GROUP BY 
    jobtitle 
ORDER BY 
    Count DESC
LIMIT 20;"""
pd.read_sql(query_intersect_union, conn)

,JobTitle,Count
0,POLICE OFFICER III,779
1,POLICE OFFICER I,467
2,POLICE OFFICER II,361
3,INSTITUTIONAL POLICE OFFICER,13


close the connection

In [ ]:
conn.close()